# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Apr 24 2023 11:44AM,261644,20,9412437,"Andelyn Biosciences, Inc.",Released
1,Apr 24 2023 11:43AM,261642,21,9412438,"OK Capsule, Inc.",Released
2,Apr 24 2023 11:41AM,261641,10,HHHH0424,Hush Hush,Released
3,Apr 24 2023 11:37AM,261632,10,PRF-58936,Bio-PRF,Released
4,Apr 24 2023 11:36AM,261635,10,PRF-59066,Bio-PRF,Released
5,Apr 24 2023 11:36AM,261635,10,PRF-59144,Bio-PRF,Released
6,Apr 24 2023 11:36AM,261635,10,PRF-59153,Bio-PRF,Released
7,Apr 24 2023 11:36AM,261635,10,PRF-59158,Bio-PRF,Released
8,Apr 24 2023 11:36AM,261635,10,PRF-59164,Bio-PRF,Released
9,Apr 24 2023 11:36AM,261635,10,PRF-59176,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
21,261639,Released,1
22,261640,Released,2
23,261641,Released,1
24,261642,Released,1
25,261644,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
261639,NaN,1.0
261640,NaN,2.0
261641,NaN,1.0
261642,NaN,1.0
261644,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
261602,0.0,1.0
261612,0.0,29.0
261618,18.0,10.0
261619,1.0,0.0
261620,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
261602,0,1
261612,0,29
261618,18,10
261619,1,0
261620,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,261602,0,1
1,261612,0,29
2,261618,18,10
3,261619,1,0
4,261620,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,261602,,1
1,261612,,29
2,261618,18,10
3,261619,1,
4,261620,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Apr 24 2023 11:44AM,261644,20,"Andelyn Biosciences, Inc."
1,Apr 24 2023 11:43AM,261642,21,"OK Capsule, Inc."
2,Apr 24 2023 11:41AM,261641,10,Hush Hush
3,Apr 24 2023 11:37AM,261632,10,Bio-PRF
4,Apr 24 2023 11:36AM,261635,10,Bio-PRF
13,Apr 24 2023 11:32AM,261640,10,"Nextsource Biotechnology, LLC"
15,Apr 24 2023 11:31AM,261639,10,"Uniderm USA, Inc"
16,Apr 24 2023 11:30AM,261638,19,"Innogenix, LLC"
40,Apr 24 2023 11:26AM,261637,10,SugarBear
41,Apr 24 2023 11:10AM,261634,15,"Brookfield Pharmaceuticals, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Apr 24 2023 11:44AM,261644,20,"Andelyn Biosciences, Inc.",,1
1,Apr 24 2023 11:43AM,261642,21,"OK Capsule, Inc.",,1
2,Apr 24 2023 11:41AM,261641,10,Hush Hush,,1
3,Apr 24 2023 11:37AM,261632,10,Bio-PRF,,1
4,Apr 24 2023 11:36AM,261635,10,Bio-PRF,,9
5,Apr 24 2023 11:32AM,261640,10,"Nextsource Biotechnology, LLC",,2
6,Apr 24 2023 11:31AM,261639,10,"Uniderm USA, Inc",,1
7,Apr 24 2023 11:30AM,261638,19,"Innogenix, LLC",,24
8,Apr 24 2023 11:26AM,261637,10,SugarBear,,1
9,Apr 24 2023 11:10AM,261634,15,"Brookfield Pharmaceuticals, LLC",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 24 2023 11:44AM,261644,20,"Andelyn Biosciences, Inc.",1,
1,Apr 24 2023 11:43AM,261642,21,"OK Capsule, Inc.",1,
2,Apr 24 2023 11:41AM,261641,10,Hush Hush,1,
3,Apr 24 2023 11:37AM,261632,10,Bio-PRF,1,
4,Apr 24 2023 11:36AM,261635,10,Bio-PRF,9,
5,Apr 24 2023 11:32AM,261640,10,"Nextsource Biotechnology, LLC",2,
6,Apr 24 2023 11:31AM,261639,10,"Uniderm USA, Inc",1,
7,Apr 24 2023 11:30AM,261638,19,"Innogenix, LLC",24,
8,Apr 24 2023 11:26AM,261637,10,SugarBear,1,
9,Apr 24 2023 11:10AM,261634,15,"Brookfield Pharmaceuticals, LLC",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 24 2023 11:44AM,261644,20,"Andelyn Biosciences, Inc.",1,
1,Apr 24 2023 11:43AM,261642,21,"OK Capsule, Inc.",1,
2,Apr 24 2023 11:41AM,261641,10,Hush Hush,1,
3,Apr 24 2023 11:37AM,261632,10,Bio-PRF,1,
4,Apr 24 2023 11:36AM,261635,10,Bio-PRF,9,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Apr 24 2023 11:44AM,261644,20,"Andelyn Biosciences, Inc.",1.0,NaN
1,Apr 24 2023 11:43AM,261642,21,"OK Capsule, Inc.",1.0,NaN
2,Apr 24 2023 11:41AM,261641,10,Hush Hush,1.0,NaN
3,Apr 24 2023 11:37AM,261632,10,Bio-PRF,1.0,NaN
4,Apr 24 2023 11:36AM,261635,10,Bio-PRF,9.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 24 2023 11:44AM,261644,20,"Andelyn Biosciences, Inc.",1.0,0.0
1,Apr 24 2023 11:43AM,261642,21,"OK Capsule, Inc.",1.0,0.0
2,Apr 24 2023 11:41AM,261641,10,Hush Hush,1.0,0.0
3,Apr 24 2023 11:37AM,261632,10,Bio-PRF,1.0,0.0
4,Apr 24 2023 11:36AM,261635,10,Bio-PRF,9.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3139572,58.0,10.0
15,1308134,36.0,18.0
16,261602,1.0,0.0
18,261619,0.0,1.0
19,261638,24.0,0.0
20,784887,52.0,0.0
21,261642,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3139572,58.0,10.0
1,15,1308134,36.0,18.0
2,16,261602,1.0,0.0
3,18,261619,0.0,1.0
4,19,261638,24.0,0.0
5,20,784887,52.0,0.0
6,21,261642,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,58.0,10.0
1,15,36.0,18.0
2,16,1.0,0.0
3,18,0.0,1.0
4,19,24.0,0.0
5,20,52.0,0.0
6,21,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,58.0
1,15,Released,36.0
2,16,Released,1.0
3,18,Released,0.0
4,19,Released,24.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20,21
Status,,,,,,,
Executing,10.0,18.0,0.0,1.0,0.0,0.0,0.0
Released,58.0,36.0,1.0,0.0,24.0,52.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20,21
0,Executing,10.0,18.0,0.0,1.0,0.0,0.0,0.0
1,Released,58.0,36.0,1.0,0.0,24.0,52.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20,21
0,Executing,10.0,18.0,0.0,1.0,0.0,0.0,0.0
1,Released,58.0,36.0,1.0,0.0,24.0,52.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()